In [2]:
import pandas as pd
import os
import re
from ipysheet import from_dataframe, to_dataframe


In [2]:
# NFCU
path = './in/transactions.CSV'
df = pd.read_csv(path)
df['Date'] = pd.to_datetime(df.Date)
df.drop("No.", axis=1, inplace=True)
df.rename(columns={"Description":"Payee"}, inplace=True)
df.to_csv('./out/nfcu.csv')

In [3]:
# Ally
path = './in/ally.csv'
df = pd.read_csv(path)

df.drop(columns=[' Time', ' Type'], inplace=True)
df.rename(columns={" Description":"Payee"}, inplace=True)

df.to_csv('./out/ally.csv')


In [4]:
# Chase
path = './in/'
for filename in os.listdir(path):
    if re.match("Chase.*.CSV",filename):
        path = path + filename
df = pd.read_csv(path)
df.drop(columns=['Transaction Date', 'Category', 'Type'], inplace=True)
df.rename(columns={'Post Date':'Date'}, inplace=True)

df.to_csv('./out/Chase.csv')


In [6]:
# Amex
path = './in/activity.csv'
df = pd.read_csv(path)

df.rename(columns={"Description":"Payee"}, inplace=True)

df.to_csv('./out/amex.csv')

In [3]:
# Full View
path = './in/'
for filename in os.listdir(path):
    if re.match("transactions_.*.csv",filename):
        path = path + filename

df = pd.read_csv(path)

df['Date'] = pd.to_datetime(df.Date)

names = df['Institution'].unique().tolist()
accts = {}
for name in names:
    inst_df = df.loc[df['Institution'] == name]
    acct_list = inst_df['Account'].unique().tolist()
    for acct in acct_list:
        acct_df = df.loc[(df['Institution'] == name) & (df['Account'] == acct) & (df['Is Hidden'] == 'No') & (df['Is Pending'] == 'No')]
        acct_df.drop(columns=['Institution', 'Account', 'Is Hidden', 'Category', 'Is Pending'], inplace=True)
        acct_df.rename(columns={'Description':'Payee'}, inplace=True)
        acct_df.to_csv(f"./out/{name + acct.replace("*","")}.csv")








/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
